In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
from ipdb import set_trace

with open('.env') as f:
    _data_pth_ = f.readline().strip()
    _data_pth_ = os.path.expanduser(_data_pth_)

### convert nominal

In [2]:
def transform_categorical(dat: pd.DataFrame):
    for c in dat.columns:
        # check categorical
        if not len(dat[[c]].std()):
            dat[[c]] = dat[[c]].fillna(dat[[c]].mode()) # fill na with the mode category
            uniques = dict([(v, i) for i, v in enumerate(set(dat[c]))])
            dat[[c]] = dat[[c]].replace(uniques)

### normalize numerical: clip outliers, fill nan, standardize

In [3]:
def normalize(dat: pd.DataFrame):
    for c in dat.columns:
        # check numerical
        if len(dat[[c]].std()):
            dat[[c]] = dat[[c]].fillna(0)
            _mean, _std = dat[[c]].mean(axis=0), dat[[c]].std(axis=0)
            # clip outliers (3 std away)
            lo, hi = _mean - 3*_std, _mean + 3*_std
            dat[[c]] = dat[[c]].clip(lo.item(), hi.item())
            # normalize
            dat[[c]] = (dat[[c]] - _mean) / _std

### preprocess and save

In [4]:
if not os.path.exists(f'{_data_pth_}/processed/'):
    os.mkdir(f'{_data_pth_}/processed/')
    
for x in ['train_transaction.csv', 'test_transaction.csv', 'test_identity.csv','train_identity.csv', ]:
    if not os.path.exists(f'{_data_pth_}/processed/{x}'):
        
        mat = pd.read_csv(f'{_data_pth_}/{x}')
        # remove ID
        if 'TransactionID' in mat.columns:
            _ids = mat['TransactionID']
            mat.drop(columns=['TransactionID'], inplace=True)
        if 'isFraud' in mat.columns:
            _labels = mat['isFraud']
            mat.drop(columns=['isFraud'], inplace=True)
        transform_categorical(mat)
        normalize(mat)
        mat = pd.concat((_ids, _labels, mat), axis=1)
        set_trace()
        mat.to_csv(f'{_data_pth_}/processed/{x}', index=False)
        del mat

> /tmp/ipykernel_33588/3138857347.py(19)<module>()
     18         set_trace()
---> 19         mat.to_csv(f'{_data_pth_}/processed/{x}', index=False)
     20         del mat

ipdb> mat
        TransactionID  isFraud  TransactionDT  TransactionAmt  ProductCD  \
0             2987000        0      -1.577985       -0.278167   0.039657   
1             2987001        0      -1.577985       -0.443327   0.039657   
2             2987002        0      -1.577970       -0.317889   0.039657   
3             2987003        0      -1.577964       -0.355520   0.039657   
4             2987004        0      -1.577962       -0.355520  -1.367161   
...               ...      ...            ...             ...        ...   
590535        3577535        0       1.827664       -0.359702   0.039657   
590536        3577536        0       1.827665       -0.399424   0.039657   
590537        3577537        0       1.827671       -0.435173   0.039657   
590538        3577538        0       1.827673       -0.